In [1]:
import bs4
import urllib3
import json

In [2]:
with open("krx_code.json") as krx:
    krx_code = json.load(krx)

In [3]:
time = []

day = "2018-02-13"
for hour in range(20,24):
    for minute in range(0, 6):
        time.append(day + "/" + day + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ ":" + str(minute) + "0.json")

day = "2018-02-14"
for hour in range(0,7):
    for minute in range(0, 6):
        time.append(day + "/" + day + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ ":" + str(minute) + "0.json")

In [13]:
diction = {}

for t in time:
    with open(t) as table:
        rank = json.load(table)
        
        for v in rank.values():
            if v == t[11:-5]:
                continue
            if diction.get(v) == None:
                diction[v] = [1]
            else:
                diction[v] = [diction.get(v)[0] + 1]

In [16]:
len(diction)

317

In [17]:
focus1 = {}
for k, v in diction.items():
    if krx_code.get(k) != None and v[0] == 1:
        focus1[k] = v[0]

In [18]:
len(focus1)

154

In [19]:
def scrape_price_dayhigh(code):
    '''
    code: string
    '''

    target = "http://finance.naver.com/item/sise_day.nhn?code=" + code + "&page=1"
    pm = urllib3.PoolManager()
    html = pm.urlopen(url=target, method="GET").data
    soup = bs4.BeautifulSoup(html, 'lxml')
    data_list = soup.find_all("tr")[2:7]
    high_list = []
    low_list = []
    for data in data_list:
        new_list = data.find_all("td",class_="num")
        high = new_list[3].text.split(",")
        high = "".join(high)
        low = new_list[4].text.split(",")
        low = "".join(low)
        high_list.append(high)
        low_list.append(low)

    return max(high_list), min(low_list)

In [20]:
less_than_15 = []

In [21]:
for k in focus1.keys():
    high, low = scrape_price_dayhigh(krx_code[k])
    print(k, high, low)
    if high != '\xa0' and low != '\xa0' and int(low) != 0:
        if 100 * float(high) / float(low) - 100 < 15:
            less_than_15.append(k)

현대건설 44250 38350
재영솔루텍 2565 2095
코스모화학 28400 23450
호텔신라 91400 80300
내츄럴엔도텍 22450 19350
한화테크윈 30950 26800
화일약품 9680 7630
대한광통신 7240 6570
포스코 ICT 8820 8310
브레인콘텐츠 926 887
에스트래픽 14700 12650
삼성제약 6000 4770
고려제약 14700 10300
한화케미칼 34400 31250
진원생명과학 8430 6840
지엘팜텍 5080 4430
코웰패션 5560 5180
바이오톡스텍 12800 10000
삼일제약 20000 12800
대우조선해양 23000 18850
대한항공 36750 34400
대한뉴팜 14300 13100
쎌바이오텍 49150 45500
레드캡투어 15650 15050
보락 1975 1375
휴마시스 2635 2175
이마트 292500 270000
대봉엘에스 15300 11900
액토즈소프트 22000 17250
상아프론테크 17550 14800
용평리조트 9720 8170
SCI평가정보 5880 4755
한일진공 3320 2880
녹십자 211000 187000
GST 12600 10950
휠라코리아 89000 81400
GS 67100 61700
CJ헬로 8870 7960
모토닉 11450 10000
삼양통상 46000 43350
안국약품 16700 15450
AJ렌터카 11600 10350
마크로젠 47650 39900
삼일 2215 2085
한국전자홀딩스 820 785
정다운 4445 4035
대우건설 5920 4790
슈프리마 28550 24000
일진머티리얼즈 35800 31550
레고켐바이오 67000 51800
리켐 2365 1830
코위버 8340 7570
와이엠티 35500 31800
유안타제2호스팩 2025 0
타이거일렉 8180 6510
앱클론 61200 54000
디에이치피코리아 11400 10000
엠에스오토텍 4150 3435
국제약품 8270 5780
지스마트글로벌 14150 

In [22]:
len(less_than_15)

98

In [23]:
json = json.dumps(less_than_15, ensure_ascii=False)
f = open("2018-02-14_focus_group.json","w")
f.write(json)
f.close()